# Causal Graph Fuzzy (MISO)

In [ ]:
!pip install --quiet git+https://github.com/PatriciaLucas/AutoML.git
!pip3 install -U git+https://github.com/PYFTS/pyFTS
!git clone https://github.com/FutureLab-DCC/CGF-LLM.git

In [2]:
from AUTODCETS import datasets
import pandas as pd
import numpy as np
import sys
sys.path.append('CGF-LLM')
import CGFuzzy, utils

In [3]:
name_dataset = 'SONDA'
target = 'glo_avg'
df = utils.get_dataset(name = name_dataset)

max_lags = 20
partitions = 30
database_path = 'CGFuzzy.db'
epochs = 20
model_name = 'CGFuzzy'

windows = utils.rolling_window(df, 10)
utils.execute("CREATE TABLE IF NOT EXISTS results(model TEXT, name_dataset TEXT, window INT, predict FLOAT, \
               real FLOAT)", database_path)

[]

In [4]:
# Loop through the windows
for i, window in enumerate(windows):
    if i < 0:
      print('window already executed')
    else:
      # Exclude constant series.
      for variable in window.columns:
          if window[variable].max() == window[variable].min():
              window = window.drop(variable, axis=1)
              print(f"Variables {variable} were deleted because they are constant.")

      idx = int(0.8 * len(window))
      train_dataset, test_dataset = window.head(idx), window.tail(len(window) - idx)

      # CGFuzzy
      ts_fuzzy, partitioners = CGFuzzy.fuzzyfy(train_dataset, npart=partitions)

      model, graph = CGFuzzy.fit(train_dataset, ts_fuzzy, target, max_lags)

      forecasts = CGFuzzy.predict(test_dataset, model, graph, max_lags, target, partitioners)
      real = test_dataset[target].squeeze().tolist()[max_lags:]
      for j in range(len(forecasts)):
          utils.execute_insert("INSERT INTO results VALUES(?, ?, ?, ?, ?)", (model_name, name_dataset, i, forecasts[j], real[j]), database_path)

      print(f'Save window: {i}')

Save window: 0
Save window: 1
Save window: 2
Save window: 3


## How to view the results

In [5]:
results = utils.get_metrics(database_path)
results

[     Model Dataset  AVG NRMSE  STD NRMSE
 0  CGFuzzy   SONDA   0.126853   0.061816]